## 1.酒店评论情感分析
我们这里解决的问题，是一个具体场景下的性感分析，准确一点说，是我们想借助自然语言处理对文本的情感分类能力，自动对酒店评论数据进行情感分析，进而可以借助情感分析的结果完成酒店的筛选。

## 2. 数据读取


### 2.1工具库

In [2]:
import warnings
warnings.filterwarnings('ignore')
import jieba
import numpy as np
import codecs  ##codecs提供的open方法来指定打开的文件的语言编码，它会在读取的时候自动转换为内部unicode 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

### 2.2 停用词

In [3]:
stopwords = []
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        stopwords.append(line.strip())

### 2.3 评论数据处理

In [4]:
import os
import sys

def get_content(fullname):
    f = codecs.open(fullname, 'r', encoding='gbk', errors='ignore')
    lines = []
    
    for eachline in f.readlines():
        eachline = eachline.strip()
        
        if eachline:  # 当前行不为空
            lines.append(eachline)
    f.close()
    return lines

# 需处理的数据路径
inp = 'E:\下载\senti_analysis-master\data\ChnSentiCorp_htl_ba_2000'
folders = ['neg', 'pos']
for foldername in folders:
    outp = '1000_' + foldername + '.txt'   # 输出文件
    output = codecs.open(outp, 'w')
    
    rootdir= os.path.join(inp, foldername)
    for each_file in os.listdir(rootdir):
        contents = get_content(os.path.join(rootdir, each_file))
        output.write(''.join(contents)+'\n')
        
    output.close()
        

### 2.4 读取评论数据

In [5]:
def read_file(in_f, sentiment, stopwords, words, sentences):
    with open(in_f, 'r', encoding='gbk') as f:
        for line in f.readlines():
            try:
                segs = jieba.lcut(line.strip())
                # 停用词过滤
                segs = [word for word in segs if word not in stopwords and len(word)>1]
                # 记录词语
                words.extend(segs)
                # 添加（分词评论，情感）的元组
                sentences.append((segs, sentiment))
            except:
                print(line)
                continue
                
# 读取数据
words = []
sentences = []
# 好评数据
sentiment = 1
read_file('1000_pos.txt', 1, stopwords, words, sentences)

# 差评数据
sentiment = 0
read_file('1000_neg.txt', 0, stopwords, words, sentences)

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\WANGZH~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.635 seconds.
Prefix dict has been built succesfully.


In [6]:
words[:10]

['距离', '川沙', '公路', '较近', '公交', '指示', '蔡陆线', '麻烦', '建议', '路线']

In [7]:
sentences[:2]

[(['距离', '川沙', '公路', '较近', '公交', '指示', '蔡陆线', '麻烦', '建议', '路线', '房间', '较为简单'],
  1),
 (['商务', '大床', '房间', '很大', '床有', '2M', '整体', '感觉', '经济', '实惠', '不错'], 1)]

## 3.数据分析

In [15]:
words_df = pd.DataFrame({'评论词语':words})
words_stat = words_df.groupby(by=['评论词语'])['评论词语'].agg({'计数':np.size})
words_stat = words_stat.reset_index().sort_values(by=['计数'], ascending=False)
words_stat.head(20)

,评论词语,计数
10533,酒店,2741
5629,房间,1899
6779,服务,950
7373,没有,875
2225,入住,764
1235,不错,758
7186,比较,552
2675,前台,509
5489,感觉,498
10930,非常,497


## 4.机器学习解决方案

In [8]:
# 切分数据
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
x = [' '.join(sentence) for sentence in x]
# 数据集划分
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.1)

In [9]:
len(x_train)  # 1800条评论作为训练集

1800

In [12]:
x_train[:3]

['酒店 比较 房间 还好 酒店 市中心 但离 出差 公司 较近 几次',
 '第二次 入住 客户 很近 图个 方便 先说 第一次 入住 空调 不冷 房间 桑拿房 归类 运气 问题 略过 不计 网络 速度慢 只能 不可思议 形容 中国电信 宽频 服务 酒店 速度 拨号上网 还慢 算是 眼界 第二次 checkin 就让 决定 酒店 第三次 前台 男员工 吊儿郎当 不说 一副 爱理不理 腔调 三句话 勉强 几个 搞不好 老板 儿子 想不出 酒店 这种 员工 原因 何在 早餐 烹饪 厨师 知道 教育 躲来躲去 想下 一碗 半天 硬是 不到 后面 服务台 小姐 索性 装作 看到 估计 希望 多一事不如少一事 房间 设备 这次 正常 赶走 一个 客人 轻而易举 办法 发现 管理 适用 员工 酒店 管理 问题 只能 酒店 觉得 根本 不想 做个 四星级 酒店 吹毛求疵 道歉 绝大多数 兢兢业业 员工 表示歉意',
 '洒店 房间 太小 设计 不合理 电梯 三十 几层楼 两个 电梯 早餐 需先到 三楼 一条 长廊 电梯 37 一次 电梯 最少 要花 分钟 价格 偏贵 早餐 宽带 费用 五星级 酒店 宾馆 反馈 2008 25 需要 更正 酒店 总共 29 早餐 27 电梯 一共 三个 房间 大小 区别 这次 所住 比较 宽带 08 已经 宽带 免费 给予 携程 用户 含早 优惠 希望 继续 支持']

In [13]:
y_train[:3]

[1, 0, 0]

In [14]:
# 特征抽取
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    ngram_range=(1,2),    # 使用长度为1和2的ngram
    max_features=1000,    # 保留最高频的1000个ngrams
)
vec.fit(x_train)

In [15]:
from sklearn.naive_bayes import MultinomialNB   # 使用朴素贝叶斯

classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [16]:
classifier.score(vec.transform(x_test), y_test)  # 模型评估

0.875

可以换成其他的分类器，如逻辑斯蒂回归、SVM等

## 5.深度学习解决方案

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import json
import pandas as pd

Using TensorFlow backend.


In [20]:
tokenizer = Tokenizer(nb_words = 2500, split=' ')
tokenizer.fit_on_texts(x)
X = tokenizer.texts_to_sequences(x)
X = pad_sequences(X)

In [22]:
# 设定embedding维度等超参数
embed_dim = 16
lstm_out = 1000
batch_size = 32   # 以32个样本为一组

# 构建LSTM网络完成感情分析
model = Sequential()
model.add(Embedding(2500, embed_dim, input_length=X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2, activation='softmax'))
# 设置损失函数、优化器和评估标准
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 显示训练进程
Y = pd.get_dummies(pd.DataFrame({'label':[str(target) for target in y]})).values
# 数据集拆分
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.1, random_state=2018)

# 拟合与训练模型
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=10)

# 结果评估
score, acc = model.evaluate(X_valid, Y_valid, verbose=2, batch_size=batch_size)
print("LogLoss损失：%.2f"%(score))
print("验证集的准确率：%.2f"%(acc))


Epoch 1/10
1800/1800 [==============================] - 336s 187ms/step - loss: 0.7784 - accuracy: 0.5339
Epoch 2/10
1800/1800 [==============================] - 401s 223ms/step - loss: 0.7421 - accuracy: 0.4983
Epoch 3/10
1800/1800 [==============================] - 406s 225ms/step - loss: 0.6836 - accuracy: 0.5372
Epoch 4/10
1800/1800 [==============================] - 411s 228ms/step - loss: 0.6345 - accuracy: 0.6461
Epoch 5/10
1800/1800 [==============================] - 410s 228ms/step - loss: 0.5739 - accuracy: 0.6972
Epoch 6/10
1800/1800 [==============================] - 424s 236ms/step - loss: 0.4451 - accuracy: 0.8167
Epoch 7/10
1800/1800 [==============================] - 465s 258ms/step - loss: 0.3576 - accuracy: 0.8461
Epoch 8/10
1800/1800 [==============================] - 452s 251ms/step - loss: 0.2963 - accuracy: 0.8772
Epoch 9/10
1800/1800 [==============================] - 458s 255ms/step - loss: 0.2244 - accuracy: 0.9172
Epoch 10/10
1800/1800 [======================

In [23]:
model.save('D:\Models\my_model_1.h5')

In [24]:
from keras.models import load_model
model2 = load_model('D:\Models\my_model_1.h5')

score, acc = model2.evaluate(X_valid, Y_valid, verbose=2, batch_size=batch_size)
print("LogLoss损失：%.2f"%(score))
print("验证集的准确率：%.2f"%(acc))

LogLoss损失：0.52
验证集的准确率：0.80


由于训练数据集很少，机器学习和深度学习的模型差距不大，后续任务是爬取更多的酒店数据，同时对比模型的准确率，最终通过一个合适的模型判别一个酒店所有的评论好坏，统计真正的好评率，因为存在打的星与实际评论不符的情况，同时还可以统计一片区域的酒店评论情况，这些是数据分析的内容，做不做随意。